# 24: Regularization

## What is Regularization?
In an attempt to fit a good model to data, we often tend to overfit. In this section, you will learn about Regularization, a technique used to avoid overfitting. Regularization discourages overly complex models by penalizing the loss function.

## Recap: Why Regularize?

### The Bias-Variance Tradeoff

When we did Linear Regression, we briefly talked about the BV Tradeoff.

![](http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png)

![](https://miro.medium.com/max/544/1*Y-yJiR0FzMgchPA-Fm5c1Q.jpeg)

1. **High bias** 
    * Systematic error in predictions (i.e. the average)
    * Bias is about the strength of assumptions the model makes
    * Underfit models tend to have high bias


2. **High variance**
    * The model is highly sensitive to changes in the data
    * Overfit models tend to have low bias and high variance
    
    
![](https://gblobscdn.gitbook.com/assets%2F-LvBP1svpACTB1R1x_U4%2F-LvNWUoWieQqaGmU_gl9%2F-LvNoby-llz4QzAK15nL%2Fimage.png?alt=media&token=41720ce9-bb66-4419-9bd8-640abf1fc415)

* Underfit Models fail to capture all of the information in the data
    * Ex. the mean leaves a lot of information on the table
* Overfit models fit to the noise in the data and fail to generalize
* How would we know if our model is over or underfit?
    * Train test split
    * Look at the testing error
    * As model complexity increases so does the possibility for overfitting

# Regularization

*The below explanation is thanks to Anuja Nagpal's awesome article: [L1 and L2 Regularization Methods](https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c) in Towards Data Science*

Regularization methods in linear models come in two types:


A regression model that uses L1 regularization technique is called Lasso Regression and model which uses L2 is called Ridge Regression.


The key difference between these two is the penalty term.


### **Ridge regression** or L2

adds “squared magnitude” of coefficient as penalty term to the loss function. Here the highlighted part represents L2 regularization element.

### Cost function
![Ridge_math](ridge_math.png)


Here, if lambda is zero then you can imagine we get back OLS. However, if lambda is very large then it will add too much weight and it will lead to under-fitting. Having said that it’s important how lambda is chosen. This technique works very well to avoid over-fitting issue.

### **Lasso Regression** (Least Absolute Shrinkage and Selection Operator) or L1

adds “absolute value of magnitude” of coefficient as penalty term to the loss function.

### Cost function
![lasso_math](lasso_math.png)

Again, if lambda is zero then we will get back OLS whereas very large value will make coefficients zero hence it will under-fit.
The key difference between these techniques is that Lasso shrinks the less important feature’s coefficient to zero thus, removing some feature altogether. So, this works well for feature selection in case we have a huge number of features.


# Why Regularize?

Remember overfitting?  One way models overfit is by becoming too complex, they may assign very different weights, or coefficients, to different features.  Both L1 and L2 regularization push weights toward zero, simplifying the model by removing or minimizing the effect of coefficients that are of less importance, thus simplifying a model.

L2 regularization also penalizes very large weights more than smaller ones.  This prevents a model from putting too much emphasis on one or a few features while ignoring others that may also be relevant.

L1 and L2 regularization can each be tried in separate models, or they can be combined in a single model.

### Standardization before Regularization

An important step before using either Lasso or Ridge regularization is to first standardize your data such that it is all on the same scale. Regularization is based on the concept of penalizing larger coefficients, so **if you have features that are on different scales, some will get unfairly penalized**. A downside of standardization is that the value of the coefficients become less interpretable and must be transformed back to their original scale if you want to interpret how a one unit change in a feature impacts the target variable.

**Scaler documentation:**

* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

## Let's Code! 

Start with a regular Linear Regression and note some important things about scikit-learn's syntax.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../resources/Housing_Prices/train.csv')

# Create X and y
y = df['SalePrice']
X = df.drop(columns=['SalePrice'], axis=1)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

# Remove "object"-type features from X
cont_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]

# Remove "object"-type features from X_train and X_test
X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

X_train_cont.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
567,568,20,70.0,10171,7,5,2004,2004,168.0,2,...,532,0,0,0,0,0,0,0,3,2010
1416,1417,190,60.0,11340,4,6,1885,1950,0.0,0,...,560,0,0,114,0,0,0,0,4,2010
1372,1373,60,75.0,9750,7,6,1998,1998,0.0,975,...,583,253,170,0,0,0,0,0,6,2006
172,173,160,44.0,5306,7,7,1987,1987,0.0,495,...,504,441,35,0,0,0,0,0,6,2006
810,811,20,78.0,10140,6,6,1974,1999,99.0,663,...,484,265,0,0,0,0,648,0,1,2006


In [3]:
# Impute missing values with median using SimpleImputer
impute = SimpleImputer(strategy='median') 
X_train_imputed = impute.fit_transform(X_train_cont) # what is fit_transform?
X_test_imputed = impute.transform(X_test_cont) # vs just .transform?


# Create X_cat which contains only the categorical variables
features_cat = [col for col in X.columns if X[col].dtype in [np.object]]
X_train_cat = X_train.loc[:, features_cat]
X_test_cat = X_test.loc[:, features_cat]

# Fill missing values with the string 'missing'
X_train_cat.fillna(value='missing', inplace=True)
X_test_cat.fillna(value='missing', inplace=True)



# Scale the train and test data
ss = StandardScaler()
X_train_imputed_scaled = ss.fit_transform(X_train_imputed)
X_test_imputed_scaled = ss.transform(X_test_imputed)

# Fit the model 
linreg_norm = LinearRegression()
linreg_norm.fit(X_train_imputed_scaled, y_train)


# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Convert these columns into a DataFrame 
columns = ohe.get_feature_names(input_features=X_train_cat.columns)
cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=columns)
cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=columns)

X_train_all = pd.concat([pd.DataFrame(X_train_imputed_scaled), cat_train_df], axis=1)
X_test_all = pd.concat([pd.DataFrame(X_test_imputed_scaled), cat_test_df], axis=1)


# Fitting the model
linreg_all = LinearRegression()
linreg_all.fit(X_train_all, y_train)

print('Training r^2:', linreg_all.score(X_train_all, y_train))
print('Test r^2:', linreg_all.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg_all.predict(X_train_all)))
print('Test MSE:', mean_squared_error(y_test, linreg_all.predict(X_test_all)))
print('Training RMSE:', mean_squared_error(y_train, linreg_all.predict(X_train_all))**0.5)
print('Test RMSE:', mean_squared_error(y_test, linreg_all.predict(X_test_all))**0.5)

Training r^2: 0.9359966077871456
Test r^2: -1.9594876655685186e+19
Training MSE: 402006558.2538813
Test MSE: 1.2501894025843791e+29
Training RMSE: 20050.10120308327
Test RMSE: 353580175149057.1


## Fitting Ridge and Lasso

* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

## Lasso Regression

In [4]:
from sklearn.linear_model import Lasso

lasso = Lasso() # Lasso is also known as the L1 norm 
lasso.fit(X_train_all, y_train)

print('Training r^2:', lasso.score(X_train_all, y_train))
print('Test r^2:', lasso.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train_all)))
print('Test MSE:', mean_squared_error(y_test, lasso.predict(X_test_all)))

Training r^2: 0.9359681086176651
Test r^2: 0.888684112594205
Training MSE: 402185562.09476894
Test MSE: 710215967.2621554


In [5]:
lasso = Lasso(alpha=10) # adjusting HYPERPARAMETERS -- check documentation!
lasso.fit(X_train_all, y_train)

print('Training r^2:', lasso.score(X_train_all, y_train))
print('Test r^2:', lasso.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train_all)))
print('Test MSE:', mean_squared_error(y_test, lasso.predict(X_test_all)))

Training r^2: 0.9343826511712741
Test r^2: 0.8966777526569275
Training MSE: 412143851.32359606
Test MSE: 659215063.9643533


In [6]:
# how many coefficients were brought to zero?

print("Total number of coefficients: ", len(lasso.coef_))
print("Coefficients close to zero: ", sum(abs(lasso.coef_) < 10**(-10)))

Total number of coefficients:  296
Coefficients close to zero:  77


## Ridge Regression

In [7]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=10) # Ridge is also known as the L2 norm
ridge.fit(X_train_all, y_train)

print('Training r^2:', ridge.score(X_train_all, y_train))
print('Test r^2:', ridge.score(X_test_all, y_test))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train_all)))
print('Test MSE:', mean_squared_error(y_test, ridge.predict(X_test_all)))

Training r^2: 0.8990002650425939
Test r^2: 0.8834542222982164
Training MSE: 634381310.5991352
Test MSE: 743583635.4522325


## Ridge & Lasso: Other benefits

### Ridge:
* We can "shrink down" prediction variables effects instead of deleting/zeroing them
* When you have features with high multicollinearity, the coefficients are automatically spread across them (you won't have redundancy)
* Since includes all features it can be computationally expensive (for many variables)

### Lasso:
* When you have a lot of variables it performs feature selection for you!
* Multicollinearity is also dealt with
